# NENA to TF

This notebook will be used to develop code for converting texts from .nena format to Text-Fabric. The parser has principally been written by Hannes Vlaardingerbroek. Many thanks to him for his hard work on it. Updates and refinements have been added by Cody Kingham.

In [1]:
import os
import sys
import collections
import re
from pathlib import Path
from tf.convert.walker import CV
from tf.fabric import Fabric

# path to parser
parserpath = f'../../nena_corpus/parse_nena/'
sys.path.append(parserpath)
from nena_parser import NenaLexer, NenaParser

# paths
VERSION = '0.01'
OUT_DIR = Path(f'../tf/{VERSION}')
data_dir = Path(f'../../nena_corpus/nena/{VERSION}')
dialect_dirs = list(Path(data_dir).glob('*'))

Parser debugging for NenaParser written to parser.out


In [2]:
TF = Fabric(locations=[str(OUT_DIR)], silent=True)
cv = CV(TF)

## Parse NENA

The NENA Parser delivers the text as structured morphemes, which can then be processed into a TF graph. We do that below by opening each source text, retrieving its parsed form, and begin each iteration. 

In [3]:
lexer = NenaLexer()
parser = NenaParser()

In [4]:
# dialect2file2parsed = collections.defaultdict(lambda: collections.defaultdict())

# nparsed = 0

# for dialect in sorted(dialect_dirs):    
#     print()
#     print(dialect.name)
#     for file in sorted(dialect.glob('*.nena')):
#         with open(file, 'r') as infile:
#             text = infile.read()
#             print(f'parsing: {file.name}')
#             parse = parser.parse(lexer.tokenize(text))
#             nparsed += 1
#             dialect2file2parsed[dialect.name][file.name] = parse
            

# print('\n', nparsed, 'texts ready for conversion')

In [5]:
# linenum, elements = dialect2file2parsed['Urmi_C']['Village Life.nena'][1][0][
# eg_morph = elements[0]

In [6]:
# dialect2file2parsed['Barwar']['A Hundred Gold Coins.nena'][1][0][0]

# Metadata

In [7]:
slotType = 'morpheme'

otext = {
    'sectionTypes': 'dialect,text,line',
    'sectionFeatures': 'dialect,title,number',
    'fmt:text-orig-full': '{text}{trailer}',
}

description = ''.join("""
The NENA linguistic corpus is derived from decades of 
field work by Geoffrey Khan and his students, 
interviewing living speakers. The NENA dialects 
remain critically endangered.
""".split('\n'))

generic = {
    'origin': 'Cambridge University, Faculty of Asian and Middle Eastern Studies',
    'author': 'Geoffrey Khan et al.',
    'editors': 'Cody Kingham, James Strachan, Hannes Vlaardingerbroek',
    'researchers': 'Paul Noordlander, Dorota Molin',
    'source': description,
}

intFeatures = {'number'}

d = 'about'

featureMeta = {
    'dialect': {d: 'name of a dialect in Northeastern Neo-Aramaic'},
    'title': {d: 'title of a text (story)'},
    'version': {d: 'the version of the story if there are multiple instances of the same story'},
    'number': {d: 'sequential number of a paragraph or line within a text or paragraph, respectively'},
    'text': {d: 'plain text representation of a morpheme or word'},
    'trailer': {d: 'space, punctuation, or other stylistic text at the end of a morpheme or word'},
    'letters': {d: 'space-separated string of letters in a morpheme or word'},
    'speaker': {d: 'speaker who interjects in the text with morphemes or words (often researcher\'s initials'},
    'footnotes': {d: 'explanatory footnotes on a morpheme or text'},
    'lang': {d: 'language of a morpheme that is foreign to the text'},
    'foreign': {d: 'indicates whether a morpheme is foreign to the text (see also lang)'},
    'comment': {d: 'an explanatory comment inserted in the text, stored on morpheme'},
    'continued_from': {d: 'text is a follow-up to the named text'},
    'informant': {d: 'name of the person who spoke these words'},
    'place': {d: 'place the text was recorded'},
    'source': {d: 'name of the file from which this text was converted'},
    'text_id': {d: 'id of a text within its original publication; can overlap between publications'},
}

# Converter

Build a TF Walker class that can walk over the NENA parsed data and fit the text graph.

In [8]:
def make_footnotes(fn_dict):
    """Format footnote dict into string"""
    if fn_dict:
        return '; '.join(
            f'[^{num}]: {txt}' for num, txt in fn_dict.items()
        )
    else:
        return None

def director(CV):
    """Walk the source data and produce a TF graph"""
    
    info = TF.tm.info
    
    for dialect_dir in sorted(dialect_dirs):  
        
        # make dialect node
        dialect = cv.node('dialect')
        dia = dialect_dir.name
        cv.feature(dialect, dialect=dia)
        
        for file in sorted(dialect_dir.glob('*.nena')):
            
            info(f'processing: [{file}]')
            
            with open(file, 'r') as infile:
                nena_text = infile.read()
            
            # parse the .nena format
            header, paragraphs = parser.parse(lexer.tokenize(nena_text))
            
            # -- begin TF node creation --
            
            # cv.node initializes a node object
            # all slots added in between its creation and 
            # termination will be considered embedded within
            # this node; same is true of following cv.node calls
            text = cv.node('text')
            cv.feature(text, **header) # adds features to supplied node
            title = header['title']
            
            for i, para in enumerate(paragraphs):
                
                # TODO: Process footnotes here
                if len(para[0]) != 2:
                    continue
                
                # make paragraph node
                paragraph = cv.node('paragraph')
                cv.feature(paragraph, number=i+1)
                
                for line_num, line_elements in para:
                    
                    # make line nodes
                    line = cv.node('line')
                    cv.feature(line, number=line_num)
                    
                    # make words by composing morphemes
                    # this must be done iteratively and 
                    # reset once a complete word has been
                    # assembled. This happens with this_word
                    # and the loop below. Other elements are 
                    # also dealt with in the loop
                    word = cv.node('word')
                    inton = cv.node('inton')
                    subsentence = cv.node('subsentence')
                    sentence = cv.node('sentence')
                    prev_morph = None
                    morph = None

                    for i, elem in enumerate(line_elements):
                        
                        end = i+1 == len(line_elements)

                        # process morphemes as slots
                        # 'slot' being the most basic element
                        if elem.__class__.__name__ == 'Morpheme':
                            
                            prev_trail = cv.get('trailer', prev_morph) if prev_morph else None
                            trail = elem.trailer
                            
                            # -- trigger word boundaries

                            # add morpheme to prev word if prev trailer is good
                            # (word remains active)
                            if prev_trail is not None and re.match('^\s*$|^[-=]\s*$', prev_trail):
                                prev_morph = morph
                                morph = cv.slot()
                                
                            # add morpheme to new word otherwise
                            # (word is terminated and replaced)
                            else:
                                prev_morph = morph if morph else None
                                morph = cv.slot()
                                
                                if prev_trail is not None:
                                    cv.terminate(word)
                                    word = cv.node('word')
                            
                            # prepare features on morphemes
                            # NB: None values are ignored by CV
                            fs = elem.__dict__
                            trailer = trail.replace('/', '\n')
                            
                            feats = {
                                'text': ''.join(fs['value']),
                                'letters': ' '.join(fs['value']),
                                'trailer': trailer,
                                'speaker': fs.get('speaker'),
                                'footnotes': make_footnotes(fs.get('footnotes', {})),
                                'lang': fs.get('lang'),
                                'foreign': str(fs.get('foreign')) if fs.get('foreign') else None,
                            }
                            
                            # add features to morpheme
                            cv.feature(morph, **feats)
                            cv.terminate(morph)
                                
                            # -- trigger linguistic node bounds --
                            
                            # intonation group bounds
                            if re.search('\u02c8', trail):
                                cv.terminate(inton)
                                if not end:
                                    inton = cv.node('inton')
                            
                            # subsentences
                            if re.search('[,;:\u2014\u2013]', trail):
                                cv.terminate(subsentence)
                                if not end:
                                    subsentence = cv.node('subsentence')
                            
                            # sentences
                            elif re.search('[.!?]', trail):
                                cv.terminate(subsentence)
                                cv.terminate(sentence)
                                if not end:
                                    subsentence = cv.node('subsentence')
                                    sentence = cv.node('sentence')
                                
                        # add other elements
                        else:
                            kind, data = elem
                            # orphaned footnotes as text feature for now
                            if kind == 'footnote':
                                cv.feature(text, footnote=make_footnotes(data))
                            # everything else as feature of prev morph
                            else:
                                cv.feature(prev_morph, **{kind:str(data)})
                    
                    # sanity check at end of line
                    test = {'inton','sentence', 'subsentence'} & cv.activeTypes()
                    if test:
                        sys.stderr.write(f'force-closing types {test} in {title} ln {line_num}\n')
                        cv.terminate(inton)
                        cv.terminate(subsentence)
                        cv.terminate(sentence)
                        
                    # -- trigger section node bounds --
                    cv.terminate(word)
                    cv.terminate(line)
                cv.terminate(paragraph)
            cv.terminate(text)
        cv.terminate(dialect)

## Test good


In [9]:
good = cv.walk(
    director,
    slotType,
    otext=otext,
    generic=generic,
    intFeatures=intFeatures,
    featureMeta=featureMeta,
    warn=True,
    force=False,
)

  0.00s Importing data from walking through the source ...
   |     0.00s Preparing metadata... 
   |     0.00s No structure nodes will be set up
   |   SECTION   TYPES:    dialect, text, line
   |   SECTION   FEATURES: dialect, title, number
   |   STRUCTURE TYPES:    
   |   STRUCTURE FEATURES: 
   |   TEXT      FEATURES:
   |      |   text-orig-full       text, trailer
   |     0.01s OK
   |     0.00s Following director... 
   |     0.00s processing: [../../nena_corpus/nena/0.01/Barwar/A Hundred Gold Coins.nena]
   |     0.05s processing: [../../nena_corpus/nena/0.01/Barwar/A Man Called Čuxo.nena]
   |     0.12s processing: [../../nena_corpus/nena/0.01/Barwar/A Tale of Two Kings.nena]
   |     0.17s processing: [../../nena_corpus/nena/0.01/Barwar/A Tale of a Prince and a Princess.nena]
   |     0.33s processing: [../../nena_corpus/nena/0.01/Barwar/Baby Leliθa.nena]
   |     0.41s processing: [../../nena_corpus/nena/0.01/Barwar/Dəmdəma.nena]
   |     0.46s processing: [../../nena_cor

force-closing types {'subsentence', 'sentence'} in A Tale of a Prince and a Princess ln 32


   |     0.81s processing: [../../nena_corpus/nena/0.01/Barwar/I Am Worth the Same as a Blind Wolf.nena]
   |     0.86s processing: [../../nena_corpus/nena/0.01/Barwar/Man Is Treacherous.nena]
   |     0.88s processing: [../../nena_corpus/nena/0.01/Barwar/Measure for Measure.nena]
   |     0.89s processing: [../../nena_corpus/nena/0.01/Barwar/Nanno and Jəndo.nena]


force-closing types {'subsentence', 'sentence'} in Gozali and Nozali ln 1
force-closing types {'subsentence', 'sentence'} in I Am Worth the Same as a Blind Wolf ln 2


   |     0.95s processing: [../../nena_corpus/nena/0.01/Barwar/Qaṭina Rescues His Nephew From Leliθa.nena]
   |     0.99s processing: [../../nena_corpus/nena/0.01/Barwar/Sour Grapes.nena]
   |     0.99s processing: [../../nena_corpus/nena/0.01/Barwar/Tales From the 1001 Nights.nena]


...

   |     8.45s processing: [../../nena_corpus/nena/0.01/Urmi_C/Village Life (4).nena]
   |     8.50s processing: [../../nena_corpus/nena/0.01/Urmi_C/Village Life (5).nena]
   |     8.55s processing: [../../nena_corpus/nena/0.01/Urmi_C/Village Life (6).nena]


force-closing types {'inton'} in Village Life (5) ln 1


   |     8.98s processing: [../../nena_corpus/nena/0.01/Urmi_C/Village Life.nena]
   |     9.11s processing: [../../nena_corpus/nena/0.01/Urmi_C/Vineyards.nena]
   |     9.13s processing: [../../nena_corpus/nena/0.01/Urmi_C/Weddings and Festivals.nena]


force-closing types {'inton'} in Village Life (6) ln 34
force-closing types {'inton'} in Village Life ln 1
force-closing types {'inton'} in Village Life ln 5
force-closing types {'inton'} in Village Life ln 18
force-closing types {'inton'} in Village Life ln 20


   |     9.22s processing: [../../nena_corpus/nena/0.01/Urmi_C/Weddings.nena]
   |     9.27s processing: [../../nena_corpus/nena/0.01/Urmi_C/When Shall I Die?.nena]
   |     9.30s processing: [../../nena_corpus/nena/0.01/Urmi_C/Women Are Stronger Than Men.nena]
   |     9.41s processing: [../../nena_corpus/nena/0.01/Urmi_C/Women Do Things Best.nena]
   |     9.51s "edge" actions: 0
   |     9.51s "feature" actions: 123172
   |     9.51s "node" actions: 116479
   |     9.51s "resume" actions: 0
   |     9.51s "slot" actions: 120148
   |     9.51s "terminate" actions: 236732
   |          2 x "dialect" node 
   |      35985 x "inton" node 
   |       2544 x "line" node 
   |     120148 x "morpheme" node  = slot type
   |        351 x "paragraph" node 
   |      16708 x "sentence" node 
   |      24528 x "subsentence" node 
   |        126 x "text" node 
   |      36235 x "word" node 
   |     236627 nodes of all types
   |     9.56s OK
   |     0.04s Removing unlinked nodes ... 
   |    